In [26]:
#pip install imbalanced-learn

In [27]:
import pandas as pd
import numpy as np

# Data Exploration

##### Create tables

In [28]:
merged_df = pd.read_csv('data/opendata-with-labels.csv', index_col=0).fillna(0)
merged_df['gender.label'] = merged_df['gender.label'].apply(lambda label: 1 if label == 'male' else 0).apply(pd.to_numeric)

merged_df.head(5)


,"playful,serious","shy,bold","cheery,sorrowful","masculine,feminine","charming,awkward","lewd,tasteful","intellectual,physical","strict,lenient","refined,rugged","trusting,suspicious",...,"picky,always down","proactive,reactive","prudish,flirtatious","innocent,jaded","touchy-feely,distant","muddy,washed","quirky,predictable","never cries,often crying",race.label,gender.label
id,,,,,,,,,,,,,,,,,,,,,
WE/1,36.4,98.8,72.3,47.3,24.2,29.3,75.4,64.7,82.1,57.2,...,60.4,74.0,83.9,78.7,70.9,33.2,31.3,31.8,white,0
WE/2,57.4,89.2,64.7,13.8,19.2,45.0,67.5,38.9,80.4,54.2,...,73.9,65.0,93.3,81.0,66.0,28.1,41.7,27.5,white,1
WE/3,26.8,57.4,9.1,92.1,27.1,66.8,44.0,74.3,34.0,12.3,...,71.4,44.9,76.8,31.4,13.3,84.4,12.7,72.8,white,0
WE/4,56.9,74.4,28.2,58.4,27.7,76.2,30.6,58.3,33.7,35.7,...,65.4,29.8,82.3,49.3,25.2,72.2,55.8,35.9,white,0
WE/5,23.6,36.0,29.4,60.6,85.8,58.8,6.8,68.0,37.4,13.3,...,78.3,56.3,36.9,34.9,30.3,52.4,39.6,75.1,black,1


In [29]:
merged_df.shape

(1462, 402)

In [30]:
merged_df_cpy = merged_df.drop(['race.label', 'gender.label'], axis = 1)

In [33]:
import imblearn
oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='all')

X_over, y_over = oversample.fit_resample(merged_df_cpy, merged_df['race.label'])

In [34]:
y_over.value_counts()

hispanic           1196
black              1196
native american    1196
indian             1196
asian              1196
mideast            1196
white              1196
Name: race.label, dtype: int64

##### Standardize index names

In [35]:
traits_columns = list(filter(lambda col: ',' in col, merged_df.columns.values))

print('Total Dual Traits:', len(traits_columns))

Total Dual Traits: 400


In [36]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

def evaluate_classifier(label, model, classifier, X, y):
    
    k_fold = KFold(n_splits=5)
    
    total_accuracy = 0
    total_tries = 0
    
    for train_index, test_index in k_fold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred, normalize=True)

        total_accuracy += accuracy
        total_tries += 1
        
    print(f'{label} Accuracy using {model}:', round(total_accuracy / total_tries, 4))


def test_classifier(label, model, classifier, X, y, test_size=0.25):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred, normalize=True)

        
    print(f'Final {label} Accuracy using {model}:', round(accuracy, 4))

    



# Predict the Gender

In [41]:
from sklearn.linear_model import LassoCV, Ridge, RidgeClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

# For gender prediction, we remove the 'masculine' label
gender_columns = list(filter(lambda trait: 'masculine' not in trait, traits_columns))
X = X_over[gender_columns].values
y = y_over.values


evaluate_classifier('Gender', 'AdaBoost', AdaBoostClassifier(), X, y)
evaluate_classifier('Gender', 'KNN - 3', KNeighborsClassifier(3), X, y)
evaluate_classifier('Gender', 'Random Forest 2^8 estimators, Gini', RandomForestClassifier(n_estimators=2**8, criterion='gini'), X, y)
evaluate_classifier('Gender', 'Random Forest 2^8 estimators, Entropy', RandomForestClassifier(n_estimators=2**8, criterion='entropy'), X, y)
evaluate_classifier('Gender', 'Logistic Regression L2', LogisticRegression(penalty='l2', max_iter=5000), X, y)
evaluate_classifier('Gender', 'Ridge Classifier, alpha 0.1', RidgeClassifier(alpha=0.1), X, y)
evaluate_classifier('Gender', 'Decision Tree, vanilla ', DecisionTreeClassifier(), X, y)

test_classifier('Gender', 'Logistic Regression L2', LogisticRegression(penalty='l2', max_iter=5000), X, y, test_size=0.25)


Gender Accuracy using AdaBoost: 0.3772
Gender Accuracy using KNN - 3: 0.5979
Gender Accuracy using Random Forest 2^8 estimators, Gini: 0.8572
Gender Accuracy using Random Forest 2^8 estimators, Entropy: 0.8572
Gender Accuracy using Logistic Regression L2: 0.8572
Gender Accuracy using Ridge Classifier, alpha 0.1: 0.5963
Gender Accuracy using Decision Tree, vanilla : 0.8572
Final Gender Accuracy using Logistic Regression L2: 0.9919


In [39]:
merged_df['gender.label'].value_counts()

1    889
0    573
Name: gender.label, dtype: int64

# Predict the Race

In [40]:
from sklearn.linear_model import LassoCV, Ridge, RidgeClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

# For gender prediction, we remove the 'masculine' label
race_columns = list(traits_columns)
#race_columns.append('gender.label')

y_encoder = LabelEncoder()

X = merged_df[race_columns].values
y = y_encoder.fit_transform(merged_df['race.label'])

evaluate_classifier('Race', 'AdaBoost', AdaBoostClassifier(), X, y)
evaluate_classifier('Race', 'KNN - 3', KNeighborsClassifier(3), X, y)
evaluate_classifier('Race', 'Random Forest 2^8 estimators, Gini', RandomForestClassifier(n_estimators=2**8, criterion='gini'), X, y)
evaluate_classifier('Race', 'Random Forest 2^8 estimators, Entropy', RandomForestClassifier(n_estimators=2**8, criterion='entropy'), X, y)
evaluate_classifier('Race', 'Logistic Regression L2, 10000 iterations', LogisticRegression(penalty='l2', max_iter=10000), X, y)
evaluate_classifier('Race', 'Ridge Classifier, alpha 0.1 ', RidgeClassifier(alpha=0.1), X, y)
evaluate_classifier('Race', 'Decision Tree, vanilla ', DecisionTreeClassifier(), X, y)

test_classifier('Race', 'Random Forest 2^8 estimators, Entropy', RandomForestClassifier(n_estimators=2**8, criterion='entropy'), X, y, test_size=0.25)
test_classifier('Race', 'Ridge Classifier, alpha 0.1 ', RidgeClassifier(alpha=0.1), X, y, test_size=0.25)


Race Accuracy using AdaBoost: 0.8256
Race Accuracy using KNN - 3: 0.7832
Race Accuracy using Random Forest 2^8 estimators, Gini: 0.842
Race Accuracy using Random Forest 2^8 estimators, Entropy: 0.8359
Race Accuracy using Logistic Regression L2, 10000 iterations: 0.8762
Race Accuracy using Ridge Classifier, alpha 0.1 : 0.8885
Race Accuracy using Decision Tree, vanilla : 0.7955
Final Race Accuracy using Random Forest 2^8 estimators, Entropy: 0.8388
Final Race Accuracy using Ridge Classifier, alpha 0.1 : 0.8825


In [ ]:
from sklearn import tree
from matplotlib import pyplot as plt

decision_tree_classifier = DecisionTreeClassifier(criterion='gini')  
decision_tree_classifier.fit(X, y)



fig = plt.figure(figsize=(100,40))
_ = tree.plot_tree(decision_tree_classifier, 
                   feature_names=race_columns,  
                   class_names=y_encoder.classes_,
                   filled=True,
                   label='all')

In [ ]:
merged_df['race.label'].value_counts()['white'] / len(merged_df['race.label'])

In [ ]:
merged_df['race.label'].value_counts()

In [50]:
len(merged_df['race.label'])

1462